In [ ]:
#VG起動プログラム

!nvidia-smi
# need reinstall cmake for pyopenjtalk
!pip uninstall -y cmake
!wget https://github.com/Kitware/CMake/releases/download/v3.22.1/cmake-3.22.1-linux-x86_64.tar.gz
!tar xf cmake-3.22.1-linux-x86_64.tar.gz
!rm cmake-3.22.1-linux-x86_64.tar.gz

import os
os.environ["PATH"] += ":/content/cmake-3.22.1-linux-x86_64/bin"

!apt-get install -y espeak
!pip install kaleido cohere openai tiktoken typing-extensions==4.5.0 fastapi python-multipart uvicorn tensorflow-probability==0.20.1 gradio==3.34.0

!git clone https://github.com/jackal-mitani/voice-generator-webui.git
%cd /notebooks/voice-generator-webui
!chmod +x setup.sh
!./setup.sh

!pip install -U gradio
!pip install httpx==0.24.1
!pip install tensorboardX
!pip install protobuf==3.20
!pip install -U lxml
!pip install -U langchain
!pip install -U pydantic
!pip install -U langchain-community
!pip install SQLAlchemy==2.0.1
!pip install faiss-gpu

# Add MIT RVC model
# ref| https://chihaya369.booth.pm/items/4701666
!mkdir -p vc/models/AISO-HOWATTO
!wget -P vc/models/AISO-HOWATTO https://huggingface.co/jkzfs/VITS_models/resolve/main/AISO-HOWATTO.pth

#【RVCモデルの追加フォルダ作成】
# '/notebooks/voice-generator-webui/vc/"RVCモデルの名前"'
#【フォルダ構造】
# RVCモデルの名前
#　|— RVCモデルの名前.pth
#　|— added.index (左記のファイル名に変更）
#　|— total_fea.npy（左記のファイル名に変更）


!python3 webui.py

Sat Feb 24 05:50:14 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:00:05.0 Off |                  Off |
| 30%   26C    P8    27W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
#チェックポイントの形状確認
import os
import torch

# 現在のカレントディレクトリを出力
print("Current Directory:", os.getcwd())

# チェックポイントファイルの絶対パスを指定
checkpoint_path = '/notebooks/voice-generator-webui/vc/models/RVCモデルの名前/RVCモデルの名前.pth'

# チェックポイントファイルをロード
cpt = torch.load(checkpoint_path, map_location='cpu')

# config の内容を表示
print("Config:", cpt['config'])

# チェックポイント内のすべてのパラメータの形状を表示
print("\nParameter Shapes:")
for param in cpt['weight']:
    print(f"{param}: {cpt['weight'][param].shape}")
    
checkpoint = torch.load('/notebooks/voice-generator-webui/vc/models/RVCモデルの名前/RVCモデルの名前.pth', map_location='cpu')
for key, value in checkpoint.items():
    if hasattr(value, 'shape'):
        print(key, value.shape)

Current Directory: /notebooks/voice-generator-webui
Config: [1025, 32, 192, 192, 768, 2, 6, 3, 0, '1', [3, 7, 11], [[1, 3, 5], [1, 3, 5], [1, 3, 5]], [10, 10, 2, 2], 512, [16, 16, 4, 4], 109, 256, 768, 40000]

Parameter Shapes:
enc_p.emb_phone.weight: torch.Size([192, 768])
enc_p.emb_phone.bias: torch.Size([192])
enc_p.emb_pitch.weight: torch.Size([256, 192])
enc_p.encoder.attn_layers.0.emb_rel_k: torch.Size([1, 21, 96])
enc_p.encoder.attn_layers.0.emb_rel_v: torch.Size([1, 21, 96])
enc_p.encoder.attn_layers.0.conv_q.weight: torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.0.conv_q.bias: torch.Size([192])
enc_p.encoder.attn_layers.0.conv_k.weight: torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.0.conv_k.bias: torch.Size([192])
enc_p.encoder.attn_layers.0.conv_v.weight: torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.0.conv_v.bias: torch.Size([192])
enc_p.encoder.attn_layers.0.conv_o.weight: torch.Size([192, 192, 1])
enc_p.encoder.attn_layers.0.conv_o.bias: torch.Size([192])


In [3]:
# インデックスファイルの形状確認
import faiss
import numpy as np

# インデックスを読み込む
index = faiss.read_index('/notebooks/voice-generator-webui/vc/models/RVCモデルの名前/added.index')

# インデックスの次元数を確認
print("Index dimension:", index.d)

# クエリベクトルを準備（ここではサンプルとしてランダムなベクトルを生成）
npy = np.random.rand(10, index.d).astype('float32')  # 10個のクエリを想定、float32にキャスト

# クエリベクトルの形状を確認
print("Query vector shape:", npy.shape)

# 検索を実行して結果を確認
D, I = index.search(npy, 5)  # 最も近い5個のエントリを検索
print("Search results:", I)

# big_npy 配列のロード（サンプルとしてランダムなデータを生成）
# 実際には、big_npy 配列のデータを読み込む必要があります
big_npy = np.random.rand(index.ntotal, index.d).astype('float32')

# big_npy 配列の形状を確認
print("Big npy shape:", big_npy.shape)

# インデックスの範囲内であることを確認
print("Indexes are within range of big npy:", np.all(I < big_npy.shape[0]))

# メモリ使用量を確認（オプショナル）
import sys
print("Memory size of big npy (MB):", sys.getsizeof(big_npy) / (1024 * 1024))

Index dimension: 768
Query vector shape: (10, 768)
Search results: [[31729 39266 39945 61627 18686]
 [23725 30426 14056 46832 39945]
 [43588    -1    -1    -1    -1]
 [10175 43207 19088 14206 37202]
 [11774 46832 23725 30426 14056]
 [10175 19088 43207 54096 27303]
 [10175 19088 43207 54096 31729]
 [10175 43207 19088 37202  5769]
 [44625 37202 14206  4521 43207]
 [14056 43207 10175 19088 23725]]
Big npy shape: (64411, 768)
Indexes are within range of big npy: True
Memory size of big npy (MB): 188.7042236328125
